In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class AspectCNN(tf.keras.Model):
    def __init__(self,
                 word_embed,
                 aspect_input,
                 kernel_num=200,
                 kernel_sizes=[3, 4],
                 n_class=4):
        super(AspectCNN, self).__init__()

        ############## 相关参数 #############
        # 词嵌入
        self.word_embed = word_embed
        self.vocab_size, self.embed_dim = word_embed.shape

        # 角度向量，每个角度都由一组单词组成
        # 角度嵌入，20个角度，每个角度表示成一个向量，然后可以分成四类
        # 如角度：位置，分成四类：方便、普通、不方便、未提及
        # 如角度：口味，分成四类：好吃、普通、难吃、未提及
        self.aspect_input = aspect_input
        self.num_aspects, _ = aspect_input.shape

        # CNN处理参数
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes

        # 四分类
        self.n_class = n_class

        ############### 相关层 #########################
        self.word_embedings = tf.keras.layers.Embedding(
            self.vocab_size, self.embed_dim, weights=[self.word_embed])

        self.aspect_fc = tf.keras.layers.Dense(self.kernel_num)

        self.convs4word = [
            tf.keras.layers.Conv1D(filters=self.kernel_num,
                                   kernel_size=k,
                                   activation='tanh')
            for k in self.kernel_sizes
        ]

        self.convs4aspect = [
            tf.keras.layers.Conv1D(filters=self.kernel_num,
                                   kernel_size=k,
                                   activation=None) for k in self.kernel_sizes
        ]

        self.fc = tf.keras.layers.Dense(self.n_class, activation='softmax')

    def get_aspect_embed(self):
        # num_aspects,aspect_words,embed_size
        aspects_embed = self.word_embedings(self.aspect_input)

        # num_aspects,embed_size
        aspects_embed = tf.reduce_mean(aspects_embed, axis=1)
        return aspects_embed

    def call(self, input_text):
        batch_size = input_text.shape[0]
        feature = self.word_embedings(input_text)  # batch,seq_len,embed_size
        aspects_embed = self.get_aspect_embed()  # num_aspects,embed_size
        aspects = self.aspect_fc(aspects_embed)  # num_aspects,kernel_num
        aspects = tf.expand_dims(aspects, 1)  # num_aspects,1,kernel_num

        x = [tf.expand_dims(conv(feature), 1) for conv in self.convs4word]
        # batch, 1, seq_len-k+1, kernel_num

        y = [
            tf.nn.relu(tf.expand_dims(conv(feature), 1) + aspects)
            for conv in self.convs4aspect
        ]
        # batch, num_aspects, seq_len-k+1, kernel_num

        x0 = [i * j for i, j in zip(x, y)]
        x0 = [tf.reshape(t, tf.constant((-1, *t.shape[2:]))) for t in x0]
        x0 = [
            tf.squeeze(tf.nn.max_pool1d(t, t.shape[1], 1, padding='VALID'), 1)
            for t in x0
        ]

        shape = np.array([batch_size, self.num_aspects,
                            self.kernel_num]).astype(np.int32)
        x0 = [tf.reshape(t, shape) for t in x0]
        x0 = tf.concat(x0, 2)
        return self.fc(x0)  # batch,num_aspects,n_class

In [3]:
word_embed = tf.constant(np.random.randn(20, 8), dtype=tf.float32)
aspect_input = tf.constant(np.random.randint(2, 15, (7, 5)), dtype=tf.int32)
kernel_num = 11

model = AspectCNN(word_embed, aspect_input, kernel_num=kernel_num)

input_text = tf.constant(np.random.randint(1, 15, (2, 1113)), dtype=tf.int32)
res = model(input_text)
print(res.shape)  # 2, 7, 4

(2, 7, 4)


In [4]:
vocab_path = "./datasets/vocab.txt"
word2id = {}
with open(vocab_path, 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:
        word, idx = line.split()
        idx = int(idx)
        word2id[word] = idx
len(word2id)

127528

In [5]:
with open('./datasets/datasets.npy','rb') as f:
    train_dataset = np.load(f)
    val_dataset = np.load(f)
    test_dataset = np.load(f)
    
with open('./datasets/labels.npy','rb') as f:
    train_labels = np.load(f)
    train_ws = np.load(f)
    
    val_labels = np.load(f)
    val_ws = np.load(f)


word_embed = np.load('./saved/word_embed.npy')

aspect_input = np.load("./datasets/aspects.npy")

In [6]:
train_labels.shape, train_dataset.shape, aspect_input.shape, val_dataset.shape

((105000, 20, 4), (105000, 1113), (20, 7), (15000, 1113))

In [7]:
train_data = tf.data.Dataset.from_tensor_slices(
    (train_dataset, train_labels, train_ws)).shuffle(buffer_size=1024)
train_data = train_data.batch(32).prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE)

val_data = tf.data.Dataset.from_tensor_slices(
    (val_dataset, val_labels, val_ws)).shuffle(buffer_size=1024)
val_data = val_data.batch(32).prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE)

In [8]:
for datas, labels, wts in train_data.take(1):
    print(datas.shape)

(32, 1113)


In [9]:
105000/32, 15000/32

(3281.25, 468.75)

In [10]:
aspect_input = tf.constant(aspect_input, dtype=tf.int32)
aspect_input

<tf.Tensor: shape=(20, 7), dtype=int32, numpy=
array([[  1,   2,   3,   4,   5,   6,   7],
       [  8,   9,  10,  11,  12,  13,  14],
       [ 15,   4,  16,  17,  18,   3,  19],
       [ 20,  21,  22,  23,  24,  25,  26],
       [ 27,  28,  29,  30,  31,  32,  33],
       [ 34,  35,  36,  37,  38,  39,  40],
       [ 41,  42,  43,  44,  45,  46,  22],
       [ 47,  48,  49,  50,  51,  52,  53],
       [ 54,  55,  56,  57,  58,  53,  59],
       [ 60,  61,  62,  63,  64,  65,  66],
       [ 67,  68,  69,  70,  71,  72,  73],
       [ 74,  75,  67,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  67,  84,  85],
       [ 86,  87,  75,  88,  89,  90,  91],
       [ 92,  93,  94,  95,  96,  97,  98],
       [ 99, 100,  90, 101, 102, 103, 104],
       [105, 106, 107, 108, 109, 110, 111],
       [112, 113, 114, 115, 116, 117, 118],
       [119, 120, 121, 122, 123, 124, 125],
       [126, 127, 128, 129, 130, 114, 112]], dtype=int32)>

In [17]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()

In [18]:
model = AspectCNN(word_embed, aspect_input)

In [19]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)

In [20]:
subjects_eng = [
    'location_traffic_convenience',
    'location_distance_from_business_district',
    'location_easy_to_find',
    'service_wait_time',
    'service_waiters_attitude',
    'service_parking_convenience',
    'service_serving_speed',
    'price_level',
    'price_cost_effective',
    'price_discount',
    'environment_decoration',
    'environment_noise',
    'environment_space',
    'environment_cleaness',
    'dish_portion',
    'dish_taste',
    'dish_look',
    'dish_recommendation',
    'others_overall_experience',
    'others_willing_to_consume_again',
]

In [21]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

In [ ]:
aspect_nums = aspect_input.shape[0]
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint,
                                     directory='./saved/models',
                                     checkpoint_name='model.ckpt',
                                     max_to_keep=4)

for epoch in range(10):
    print(f"Epoch {epoch} :")

    print("Starting Training")
    for idx, data_batch in tqdm(enumerate(train_data)):
        inputs, labels, labels_ws = data_batch

        with tf.GradientTape() as tape:
            preds = model(inputs)
            loss = loss_fn(labels, preds, sample_weight=labels_ws)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

        if (idx + 1) % 200 == 0:
            labels = np.argmax(labels, axis=-1)
            preds = np.argmax(preds, axis=-1)

            f1, acc = 0, 0
            f1_details, acc_details = [], []
            for col_idx in range(aspect_nums):
                crt_f1 = f1_score(labels[:, col_idx],
                                  preds[:, col_idx],
                                  average=None)
                crt_mean_f1 = np.mean(crt_f1)

                crt_acc = accuracy_score(labels[:, col_idx], preds[:, col_idx])

                f1_details.append(crt_mean_f1)
                f1 += crt_mean_f1

                acc_details.append(crt_acc)
                acc += crt_acc
            f1 = f1 / aspect_nums
            acc = acc / aspect_nums
            f1_details = np.round(f1_details, 3)
            acc_details = np.round(acc_details, 3)

            print(f"Epoch {epoch}, Step {idx+1} on Training :")
            print(f"Traning Loss : {loss.numpy()}")
            print('Total_f1={},\n f1_details={}'.format(f1, f1_details))
            print('Total_acc={},\n acc_details={}'.format(acc, acc_details))
            print()

    print('--' * 40)
    print("Starting Validation:")
    f1_details, acc_details = [0] * aspect_nums, [0] * aspect_nums
    for idx, data_batch in tqdm(enumerate(val_data)):
        inputs, labels, labels_ws = data_batch

        preds = model(inputs)
        loss = loss_fn(labels, preds, sample_weight=labels_ws)

        labels = np.argmax(labels, axis=-1)
        preds = np.argmax(preds, axis=-1)

        for col_idx in range(aspect_nums):
            crt_f1 = f1_score(labels[:, col_idx],
                              preds[:, col_idx],
                              average=None)
            crt_mean_f1 = np.mean(crt_f1)

            crt_acc = accuracy_score(labels[:, col_idx], preds[:, col_idx])

            f1_details[col_idx] += crt_mean_f1

            acc_details[col_idx] += crt_acc

    f1_details = (np.array(f1_details) / (idx + 1)).round(3)
    f1 = np.mean(f1_details)

    acc_details = (np.array(acc_details) / (idx + 1)).round(3)
    acc = np.mean(acc)

    print(f"Epoch {epoch}, on Validation :")
    print('Total_f1={}, \n f1_details={}'.format(f1, f1_details))
    print('Total_acc={}, \n acc_details={}'.format(acc, acc_details))
    
    manager.save()

    print("==" * 40)

Epoch 0 :
Starting Training


200it [01:01,  3.22it/s]

Epoch 0, Step 200 on Training :
Traning Loss : 0.14493535459041595
Total_f1=0.31272046566029144,
 f1_details=[0.418 0.484 0.279 0.305 0.263 0.323 0.317 0.244 0.299 0.198 0.487 0.386
 0.255 0.198 0.152 0.379 0.209 0.305 0.534 0.219]
Total_acc=0.7203125,
 acc_details=[0.719 0.938 0.719 0.844 0.5   0.938 0.906 0.531 0.812 0.656 0.719 0.781
 0.594 0.656 0.438 0.688 0.719 0.844 0.625 0.781]



400it [02:02,  3.14it/s]

Epoch 0, Step 400 on Training :
Traning Loss : 0.1188097819685936
Total_f1=0.47216504171490364,
 f1_details=[0.733 0.458 0.233 0.483 0.502 1.    0.279 0.354 0.214 0.673 0.589 0.548
 0.362 0.74  0.34  0.443 0.272 0.292 0.448 0.48 ]
Total_acc=0.75625,
 acc_details=[0.938 0.844 0.875 0.906 0.625 1.    0.719 0.625 0.75  0.844 0.812 0.844
 0.688 0.75  0.438 0.594 0.688 0.781 0.781 0.625]



...

1000it [05:05,  3.25it/s]

Epoch 2, Step 1000 on Training :
Traning Loss : 0.07655569911003113
Total_f1=0.724098418432773,
 f1_details=[0.881 0.782 0.722 1.    0.9   0.5   0.658 0.732 0.766 0.93  0.768 0.902
 0.485 0.61  0.75  0.72  0.362 0.857 0.524 0.633]
Total_acc=0.8890625,
 acc_details=[0.938 0.844 0.938 1.    0.969 0.969 0.938 0.812 0.875 0.938 0.875 0.875
 0.75  0.906 0.844 0.812 0.812 0.906 0.875 0.906]



1200it [06:07,  3.24it/s]

Epoch 2, Step 1200 on Training :
Traning Loss : 0.06900061666965485
Total_f1=0.7162517609058618,
 f1_details=[0.592 0.855 0.865 0.237 0.791 0.716 1.    0.833 0.878 0.597 0.662 0.728
 0.929 0.564 0.491 0.858 0.267 0.877 0.867 0.718]
Total_acc=0.8703125,
 acc_details=[0.906 0.906 0.875 0.875 0.875 0.938 1.    0.844 0.906 0.844 0.781 0.844
 0.969 0.875 0.719 0.875 0.719 0.938 0.875 0.844]



1332it [06:47,  3.24it/s]

In [239]:
model = AspectCNN(word_embed, aspect_input)
checkpoint = tf.train.Checkpoint(model=model) # 键名一样
checkpoint.restore(tf.train.latest_checkpoint('./saved/models'))
model

In [240]:
import os
model_version = "0001"
model_name = "dianping_classifier"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path, signatures={"call", model.call})

ValueError: Expected a TensorFlow function to generate a signature for, but got {'call', <tensorflow.python.eager.def_function.Function object at 0x7f532ccfce50>}. Only `tf.functions` with an input signature or concrete functions can be used as a signature.

In [187]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'


In [188]:
!saved_model_cli show --dir {model_path} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"


In [173]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

Defined Functions:

In [28]:
import tensorflow as tf

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(100, activation='relu')
        self.dense2 = tf.keras.layers.Dense(5, activation='softmax')        
        
    @tf.function(input_signature=[tf.TensorSpec([None, 28, 28, 1], tf.float32)])
    def call(self, inputs):
        
        x = tf.reshape(inputs, [inputs.shape[0], -1])
        x = self.dense1(x)
        y = self.dense2(x)
        return y

In [30]:
my_model = MyModel()

inputs = tf.random.uniform([3, 28, 28, 1])
my_model(inputs)




TypeError: in user code:

    <ipython-input-29-cf20a7858b98>:10 call  *
        x = tf.reshape(inputs, [inputs.shape[0], -1])
    /home/yangbin7/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:193 reshape  **
        result = gen_array_ops.reshape(tensor, shape, name)
    /home/yangbin7/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/gen_array_ops.py:8087 reshape
        "Reshape", tensor=tensor, shape=shape, name=name)
    /home/yangbin7/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:473 _apply_op_helper
        raise err
    /home/yangbin7/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:470 _apply_op_helper
        preferred_dtype=default_dtype)
    /home/yangbin7/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:1341 convert_to_tensor
        ret = conversion_func(value, dtype=dtype, name=name, as_ref=as_ref)
    /home/yangbin7/anaconda3/lib/python3.7

In [5]:
tf.saved_model.save(my_model, "jjjj", signatures={"call": my_model.call})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: jjjj/assets


In [6]:
!saved_model_cli show --dir "jjjj" --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['call']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: call_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callab